In [ ]:
'''
                                                            # MIT em Engenharia de Dados: Big Data
                                                            # Análise de dados com Spark

Professor: Andre Ormastroni
Aluno: Charles de Araujo Melo


'''

In [ ]:
# Minha base de dados é de uma loja de venda de bicicletas onde utilizarei para identificar vantagens
# empresariais para futuro aumento de vendas e conhecer melhor as características dos clientes para maiores captações.

# Instalando o pyspark

In [ ]:
!pip install pyspark

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving demogra_cliente.csv to demogra_cliente.csv
Saving end_cliente.csv to end_cliente.csv
Saving transacoes.csv to transacoes.csv


In [ ]:
#Importando os módulos

from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max, when, isnan

In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *

In [26]:
#crie ou recupere sua spark session

spark = SparkSession.builder.appName("Trabalho").master('local[*]').getOrCreate()

In [ ]:
#Importando os arquivos transacoes.csv, end_cliente.csv e demogra_cliente.csv do dataset

df_transacoes = spark.read.csv(path='transacoes.csv',header=True,inferSchema=True)
df_end_cliente = spark.read.csv(path='end_cliente.csv',header=True,inferSchema=True)
df_demogra_cliente = spark.read.csv(path='demogra_cliente.csv',header=True,inferSchema=True)

In [ ]:
# Conferindo as primeiras linhas

In [ ]:
df_transacoes.show()

+--------------+----------+-----------+----+---------+---+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|transaction_id|product_id|customer_id| Ano|      Mês|Dia|pedido_online|status_pedido|         marca|linha_produto|classe_produto|tamanho_produto|preco_lista|custo_padrao|
+--------------+----------+-----------+----+---------+---+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|             1|         2|       2950|2017|fevereiro| 25|        false|     Approved|         Solex|     Standard|        medium|         medium|       7149|        5362|
|             2|         3|       3120|2017|     maio| 21|         true|     Approved| Trek Bicycles|     Standard|        medium|          large|     209147|       38892|
|             3|        37|        402|2017|  outubro| 16|        false|     Approved|    OHM Cycles|     Standard|           low|         m

In [ ]:
df_end_cliente.show()

+-----------+--------------------+---------------+---------+-------------+
|customer_id|            endereco|         estado|     pais|codigo_postal|
+-----------+--------------------+---------------+---------+-------------+
|          1|  060 Morning Avenue|New South Wales|Australia|         2016|
|          2| 6 Meadow Vale Court|New South Wales|Australia|         2153|
|          4|  0 Holy Cross Court|            QLD|Australia|         4211|
|          5| 17979 Del Mar Point|New South Wales|Australia|         2448|
|          6|    9 Oakridge Court|            VIC|Australia|         3216|
|          7|    4 Delaware Trail|New South Wales|Australia|         2210|
|          8| 49 Londonderry Lane|New South Wales|Australia|         2650|
|          9|     97736 7th Trail|New South Wales|Australia|         2023|
|         11|93405 Ludington Park|            VIC|Australia|         3044|
|         12|44339 Golden Leaf...|            QLD|Australia|         4557|
|         13| 2 Sutherlan

In [ ]:
df_demogra_cliente.show()

+-----------+--------------+---------+------+-------------------------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|customer_id|          nome|sobrenome|  sexo|compra_bike_ultimo_3_anos| Ano|      Mês|Dia|               cargo|   categoria_cargo|         segmento|possui_carro|falecido|
+-----------+--------------+---------+------+-------------------------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|          1|       Laraine|Medendorp|Female|                       93|1953|  outubro| 12| Executive Secretary|            Health|    Mass Customer|         Yes|       N|
|          2|           Eli|  Bockman|  Male|                       81|1980| dezembro| 16|Administrative Of...|Financial Services|    Mass Customer|         Yes|       N|
|          3|         Arlin|   Dearle|  Male|                       61|1954|  janeiro| 20|  Recruiting Manager|          Property|    Mass Custom

In [ ]:
# Verificando as descrições de cada tabela

In [ ]:
df_transacoes.describe().show()

+-------+-----------------+------------------+------------------+------+--------+------------------+-------------+--------------+-------------+--------------+---------------+-----------------+--------------------+
|summary|   transaction_id|        product_id|       customer_id|   Ano|     Mês|               Dia|status_pedido|         marca|linha_produto|classe_produto|tamanho_produto|      preco_lista|        custo_padrao|
+-------+-----------------+------------------+------------------+------+--------+------------------+-------------+--------------+-------------+--------------+---------------+-----------------+--------------------+
|  count|            17564|             17564|             17564| 17564|   17564|             17564|        17564|         17367|        17367|         17367|          17367|            17564|               17367|
|   mean|9997.536267365065| 45.47102026873149|1738.8473012981099|2017.0|    NULL|15.520951947164654|         NULL|          NULL|         NULL| 

In [ ]:
df_end_cliente.describe().show()

+-------+------------------+--------------------+--------+---------+-----------------+
|summary|       customer_id|            endereco|  estado|     pais|    codigo_postal|
+-------+------------------+--------------------+--------+---------+-----------------+
|  count|              3999|                3999|    3999|     3999|             3999|
|   mean|2003.9879969992498|                NULL|    NULL|     NULL|2985.755938984746|
| stddev|1154.5769124572676|                NULL|    NULL|     NULL|844.8783644255834|
|    min|                 1|          0 3rd Road|     NSW|Australia|             2000|
|    max|              4003|9993 Mccormick St...|Victoria|Australia|             4883|
+-------+------------------+--------------------+--------+---------+-----------------+



In [ ]:
df_demogra_cliente.describe().show()

+-------+------------------+------+---------+------+-------------------------+------------------+--------+-----------------+-------------------+---------------+-----------------+------------+--------+
|summary|       customer_id|  nome|sobrenome|  sexo|compra_bike_ultimo_3_anos|               Ano|     Mês|              Dia|              cargo|categoria_cargo|         segmento|possui_carro|falecido|
+-------+------------------+------+---------+------+-------------------------+------------------+--------+-----------------+-------------------+---------------+-----------------+------------+--------+
|  count|              4000|  4000|     3875|  4000|                     4000|              3913|    3913|             3913|               3494|           4000|             4000|        4000|    4000|
|   mean|            2000.5|  NULL|     NULL|  NULL|                    48.89|1977.0306670074112|    NULL|15.76463071811909|               NULL|           NULL|             NULL|        NULL|    N

In [ ]:
df_transacoes.summary().show()

+-------+-----------------+------------------+------------------+------+--------+------------------+-------------+--------------+-------------+--------------+---------------+-----------------+--------------------+
|summary|   transaction_id|        product_id|       customer_id|   Ano|     Mês|               Dia|status_pedido|         marca|linha_produto|classe_produto|tamanho_produto|      preco_lista|        custo_padrao|
+-------+-----------------+------------------+------------------+------+--------+------------------+-------------+--------------+-------------+--------------+---------------+-----------------+--------------------+
|  count|            17564|             17564|             17564| 17564|   17564|             17564|        17564|         17367|        17367|         17367|          17367|            17564|               17367|
|   mean|9997.536267365065| 45.47102026873149|1738.8473012981099|2017.0|    NULL|15.520951947164654|         NULL|          NULL|         NULL| 

In [ ]:
df_end_cliente.summary().show()

+-------+------------------+--------------------+--------+---------+-----------------+
|summary|       customer_id|            endereco|  estado|     pais|    codigo_postal|
+-------+------------------+--------------------+--------+---------+-----------------+
|  count|              3999|                3999|    3999|     3999|             3999|
|   mean|2003.9879969992498|                NULL|    NULL|     NULL|2985.755938984746|
| stddev|1154.5769124572676|                NULL|    NULL|     NULL|844.8783644255834|
|    min|                 1|          0 3rd Road|     NSW|Australia|             2000|
|    25%|              1004|                NULL|    NULL|     NULL|             2200|
|    50%|              2004|                NULL|    NULL|     NULL|             2768|
|    75%|              3004|                NULL|    NULL|     NULL|             3750|
|    max|              4003|9993 Mccormick St...|Victoria|Australia|             4883|
+-------+------------------+---------------

In [ ]:
df_demogra_cliente.summary().show()

+-------+------------------+------+---------+------+-------------------------+------------------+--------+-----------------+-------------------+---------------+-----------------+------------+--------+
|summary|       customer_id|  nome|sobrenome|  sexo|compra_bike_ultimo_3_anos|               Ano|     Mês|              Dia|              cargo|categoria_cargo|         segmento|possui_carro|falecido|
+-------+------------------+------+---------+------+-------------------------+------------------+--------+-----------------+-------------------+---------------+-----------------+------------+--------+
|  count|              4000|  4000|     3875|  4000|                     4000|              3913|    3913|             3913|               3494|           4000|             4000|        4000|    4000|
|   mean|            2000.5|  NULL|     NULL|  NULL|                    48.89|1977.0306670074112|    NULL|15.76463071811909|               NULL|           NULL|             NULL|        NULL|    N

In [ ]:
# Removendo as colunas que não serão utilizadas

In [ ]:
dft = df_transacoes.drop('Dia')
dfe = df_end_cliente.drop('codigo_postal')
dfd = df_demogra_cliente.drop('compra_bike_ultimo_3_anos')

In [ ]:
dft.show()

+--------------+----------+-----------+----+---------+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|transaction_id|product_id|customer_id| Ano|      Mês|pedido_online|status_pedido|         marca|linha_produto|classe_produto|tamanho_produto|preco_lista|custo_padrao|
+--------------+----------+-----------+----+---------+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|             1|         2|       2950|2017|fevereiro|        false|     Approved|         Solex|     Standard|        medium|         medium|       7149|        5362|
|             2|         3|       3120|2017|     maio|         true|     Approved| Trek Bicycles|     Standard|        medium|          large|     209147|       38892|
|             3|        37|        402|2017|  outubro|        false|     Approved|    OHM Cycles|     Standard|           low|         medium|     179343|      

In [ ]:
dfe.show()

+-----------+--------------------+---------------+---------+
|customer_id|            endereco|         estado|     pais|
+-----------+--------------------+---------------+---------+
|          1|  060 Morning Avenue|New South Wales|Australia|
|          2| 6 Meadow Vale Court|New South Wales|Australia|
|          4|  0 Holy Cross Court|            QLD|Australia|
|          5| 17979 Del Mar Point|New South Wales|Australia|
|          6|    9 Oakridge Court|            VIC|Australia|
|          7|    4 Delaware Trail|New South Wales|Australia|
|          8| 49 Londonderry Lane|New South Wales|Australia|
|          9|     97736 7th Trail|New South Wales|Australia|
|         11|93405 Ludington Park|            VIC|Australia|
|         12|44339 Golden Leaf...|            QLD|Australia|
|         13| 2 Sutherland Street|            VIC|Australia|
|         14|     9 Mcbride Trail|New South Wales|Australia|
|         15|9861 New Castle A...|New South Wales|Australia|
|         16|    52 Mola

In [ ]:
dfd.show()

+-----------+--------------+---------+------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|customer_id|          nome|sobrenome|  sexo| Ano|      Mês|Dia|               cargo|   categoria_cargo|         segmento|possui_carro|falecido|
+-----------+--------------+---------+------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|          1|       Laraine|Medendorp|Female|1953|  outubro| 12| Executive Secretary|            Health|    Mass Customer|         Yes|       N|
|          2|           Eli|  Bockman|  Male|1980| dezembro| 16|Administrative Of...|Financial Services|    Mass Customer|         Yes|       N|
|          3|         Arlin|   Dearle|  Male|1954|  janeiro| 20|  Recruiting Manager|          Property|    Mass Customer|         Yes|       N|
|          4|        Talbot|     NULL|  Male|1961|  outubro|  3|                NULL|                IT|    Mass Customer|        

In [ ]:
dft.count()

17564

In [ ]:
dfe.count()

3999

In [ ]:
dfd.count()

4000

In [ ]:
dft2 = dft.dropna()

In [ ]:
dfe2 = dfe.dropna()

In [ ]:
dfd2 = dfd.dropna()

In [ ]:
dft2.count()

17060

In [ ]:
dfe2.count()

3999

In [ ]:
dfd2.count()

3311

In [ ]:
dft2.show()

+--------------+----------+-----------+----+---------+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|transaction_id|product_id|customer_id| Ano|      Mês|pedido_online|status_pedido|         marca|linha_produto|classe_produto|tamanho_produto|preco_lista|custo_padrao|
+--------------+----------+-----------+----+---------+-------------+-------------+--------------+-------------+--------------+---------------+-----------+------------+
|             1|         2|       2950|2017|fevereiro|        false|     Approved|         Solex|     Standard|        medium|         medium|       7149|        5362|
|             2|         3|       3120|2017|     maio|         true|     Approved| Trek Bicycles|     Standard|        medium|          large|     209147|       38892|
|             3|        37|        402|2017|  outubro|        false|     Approved|    OHM Cycles|     Standard|           low|         medium|     179343|      

In [ ]:
dfe2.show()

+-----------+--------------------+---------------+---------+
|customer_id|            endereco|         estado|     pais|
+-----------+--------------------+---------------+---------+
|          1|  060 Morning Avenue|New South Wales|Australia|
|          2| 6 Meadow Vale Court|New South Wales|Australia|
|          4|  0 Holy Cross Court|            QLD|Australia|
|          5| 17979 Del Mar Point|New South Wales|Australia|
|          6|    9 Oakridge Court|            VIC|Australia|
|          7|    4 Delaware Trail|New South Wales|Australia|
|          8| 49 Londonderry Lane|New South Wales|Australia|
|          9|     97736 7th Trail|New South Wales|Australia|
|         11|93405 Ludington Park|            VIC|Australia|
|         12|44339 Golden Leaf...|            QLD|Australia|
|         13| 2 Sutherland Street|            VIC|Australia|
|         14|     9 Mcbride Trail|New South Wales|Australia|
|         15|9861 New Castle A...|New South Wales|Australia|
|         16|    52 Mola

In [ ]:
dfd2.show()

+-----------+--------------+---------+------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|customer_id|          nome|sobrenome|  sexo| Ano|      Mês|Dia|               cargo|   categoria_cargo|         segmento|possui_carro|falecido|
+-----------+--------------+---------+------+----+---------+---+--------------------+------------------+-----------------+------------+--------+
|          1|       Laraine|Medendorp|Female|1953|  outubro| 12| Executive Secretary|            Health|    Mass Customer|         Yes|       N|
|          2|           Eli|  Bockman|  Male|1980| dezembro| 16|Administrative Of...|Financial Services|    Mass Customer|         Yes|       N|
|          3|         Arlin|   Dearle|  Male|1954|  janeiro| 20|  Recruiting Manager|          Property|    Mass Customer|         Yes|       N|
|          5|Sheila-kathryn|   Calton|Female|1977|     maio| 13|       Senior Editor|               n/a|Affluent Customer|        

In [ ]:
dft2.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mês: string (nullable = true)
 |-- pedido_online: boolean (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- linha_produto: string (nullable = true)
 |-- classe_produto: string (nullable = true)
 |-- tamanho_produto: string (nullable = true)
 |-- preco_lista: integer (nullable = true)
 |-- custo_padrao: long (nullable = true)



In [ ]:
dfe2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- pais: string (nullable = true)



In [ ]:
dfd2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- sobrenome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mês: string (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- cargo: string (nullable = true)
 |-- categoria_cargo: string (nullable = true)
 |-- segmento: string (nullable = true)
 |-- possui_carro: string (nullable = true)
 |-- falecido: string (nullable = true)



In [ ]:
shape = dft2.count(), len(dft2.columns)
shape

(17060, 13)

In [ ]:
dft2.select('product_id', 'Mês', 'classe_produto', 'marca', 'preco_lista').show()

+----------+---------+--------------+--------------+-----------+
|product_id|      Mês|classe_produto|         marca|preco_lista|
+----------+---------+--------------+--------------+-----------+
|         2|fevereiro|        medium|         Solex|       7149|
|         3|     maio|        medium| Trek Bicycles|     209147|
|        37|  outubro|           low|    OHM Cycles|     179343|
|        88|   agosto|        medium|Norco Bicycles|     119846|
|        78|  outubro|        medium|Giant Bicycles|      17653|
|        25|    março|        medium|Giant Bicycles|     153899|
|        22|    abril|        medium|      WeareA2B|       6034|
|        15|    julho|        medium|      WeareA2B|     129284|
|        67|   agosto|        medium|         Solex|     107123|
|        12|   agosto|        medium|      WeareA2B|     123115|
|         5|  janeiro|           low| Trek Bicycles|      57464|
|        61|  janeiro|           low|    OHM Cycles|       7116|
|        35|fevereiro|   

In [ ]:
# Fazendo Join nas tabelas

In [ ]:
df_full = dfd2.join(dft2, on='customer_id', how='inner')


In [ ]:
df_full.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- sobrenome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mês: string (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- cargo: string (nullable = true)
 |-- categoria_cargo: string (nullable = true)
 |-- segmento: string (nullable = true)
 |-- possui_carro: string (nullable = true)
 |-- falecido: string (nullable = true)
 |-- transaction_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mês: string (nullable = true)
 |-- pedido_online: boolean (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- linha_produto: string (nullable = true)
 |-- classe_produto: string (nullable = true)
 |-- tamanho_produto: string (nullable = true)
 |-- preco_lista: integer (nullable = true)
 |-- custo_padrao: long (nullable = true)



In [ ]:
# 1. Visualizar a classe dos produtos comprados de todos os clientes que compraram bicicletas com o cargo de ‘Teacher’ e que possuem carro.

In [ ]:
df_full.filter((col("cargo") == 'Teacher')  & (col("possui_carro") == 'Yes')) .select('cargo','classe_produto','possui_carro').show()

+-------+--------------+------------+
|  cargo|classe_produto|possui_carro|
+-------+--------------+------------+
|Teacher|        medium|         Yes|
|Teacher|          high|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|          high|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|          high|         Yes|
|Teacher|          high|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|           low|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
|Teacher|        medium|         Yes|
+-------+--------------+------------+
only showing top 20 rows



In [ ]:
# 2. Soma de valores gastos por sexo

In [ ]:
(
    df_full
    .groupby("sexo")
    .agg({'preco_lista': 'sum'})
    .show()
)

+------+----------------+
|  sexo|sum(preco_lista)|
+------+----------------+
|   Gay|          724517|
|Female|       647866089|
|  Male|       621819105|
+------+----------------+



In [ ]:
# 3. Visualizar a soma da quantidade de compras feita pelo sexo masculino no dia 10.

In [ ]:
df_full.filter((col("Dia") == 10)  & (col("sexo") == "Male")) .select('Dia','sexo').groupBy("sexo").count().orderBy(col("count")).show()

+----+-----+
|sexo|count|
+----+-----+
|Male|  221|
+----+-----+



In [ ]:
# 4. Selecionar nomes e sobrenomes de 50 clientes que compraram bicicletas da classe ‘High’.

In [ ]:
df_full.filter((col("classe_produto") == 'high')) .select('nome','sobrenome','classe_produto').show(50)

+----------+-------------+--------------+
|      nome|    sobrenome|classe_produto|
+----------+-------------+--------------+
|      Coop|         Skey|          high|
|     Reine|       Semken|          high|
|    Faythe|        Rance|          high|
|     Janot|     McGougan|          high|
|  Langsdon|    Tranfield|          high|
| Annamarie|       Bouzan|          high|
| Madelaine|    Siegertsz|          high|
|  L;urette|       Annott|          high|
|       Lyn|       Luquet|          high|
|      Emma|        Mates|          high|
|  Leontyne|      Jackett|          high|
|     Ambur|      Symcoxe|          high|
|   Kendall|        Humes|          high|
|      Tony|      Shirley|          high|
|    Andrei|    Mulchrone|          high|
|   Pacorro| De Clairmont|          high|
|      Harv|        Pavey|          high|
|  Townsend|        Trobe|          high|
|    Jazmin|      Neumann|          high|
|    Tracey|       Verdun|          high|
|   Bentley|  Fortesquieu|        

In [ ]:
# 5. Selecionar nomes das 50 clientes do sexo ‘female’ que compraram bicicletas da classe ‘High’.

In [ ]:
df_full.filter((col("sexo") == 'Female')  & (col("classe_produto") == "high")) .select('nome','sexo','classe_produto').orderBy(col("nome")).show(50)

+-----------+------+--------------+
|       nome|  sexo|classe_produto|
+-----------+------+--------------+
|     Aarika|Female|          high|
|    Abagail|Female|          high|
|      Abbey|Female|          high|
|       Abra|Female|          high|
|       Abra|Female|          high|
|       Abra|Female|          high|
|       Addi|Female|          high|
|       Addi|Female|          high|
|       Addi|Female|          high|
|       Adey|Female|          high|
|      Adrea|Female|          high|
|   Adrianne|Female|          high|
|     Aeriel|Female|          high|
|     Agathe|Female|          high|
|       Aggi|Female|          high|
|    Agnella|Female|          high|
|       Aili|Female|          high|
|     Ailsun|Female|          high|
|      Ailyn|Female|          high|
|    Ainsley|Female|          high|
|  Albertine|Female|          high|
|  Albertine|Female|          high|
|  Albertine|Female|          high|
|Alejandrina|Female|          high|
|     Alexia|Female|        

#                              Trabalho de Classificação iris





In [23]:
from google.colab import files
uploaded = files.upload()

Saving Iris.csv to Iris (2).csv


## Análise dos dados
– A partir das caracteristicas da flor Iris, o objetivo é classificar (definir a classe / rótulo) de qual é o tipo dessa flor, entre três tipos possíveis: Versicolor, Setosa e Virginica.
- Este dataset é composto por cinco características: SepalLength (Comprimento da Sépala), SepalWidth (Largura da Sépala), PetalLength (Comprimento da Pétala), PetalWidth (Largura da Pétala) e Espécie (Species).
Objetivo
Treinar um modelo para prever e classificar uma rosa a partir das medidas fornecidas em função de suas features (SepalLengthCm, SepalWidthCm, PetalLengthCm, etc.)

# Objetivo
Treinar um modelo para prever e classificar uma rosa a partir das medidas fornecidas em função de suas features (SepalLengthCm, SepalWidthCm, PetalLengthCm, etc.)

In [24]:
!pip install pyspark


In [27]:
df_iris = spark.read.csv(path="Iris.csv", header=True, inferSchema=True)
df_iris.show()

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
|          5.4|         3.7|          1.5|         0.2|Iris-setosa|
|          4.8|         3.4|          1.6|      

In [28]:
df_iris.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [29]:
#Trabalhando com vetores

In [30]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer

In [ ]:
#Dropando linhas nulas e transformando Atributos Categóricos

In [31]:
df_iris = df_iris.na.drop()
df_iris.count()

150

In [32]:
label_index = StringIndexer(inputCol="Species", outputCol="indexedLabel")
model_label = label_index.fit(df_iris)
df_iris = model_label.transform(df_iris)
df_iris.show(50)

+-------------+------------+-------------+------------+-----------+------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|indexedLabel|
+-------------+------------+-------------+------------+-----------+------------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|         0.0|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|         0.0|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|         0.0|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|         0.0|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|         0.0|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|         0.0|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|         0.0|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|         0.0|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|         0.0|
|          4.9|         3.1|

In [33]:
df_iris.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)
 |-- indexedLabel: double (nullable = false)



In [ ]:
# Montando os vetores

In [34]:
assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"],outputCol="features")
df_features = assembler.transform(df_iris)
df_features.show(20, False)

+-------------+------------+-------------+------------+-----------+------------+-----------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |indexedLabel|features         |
+-------------+------------+-------------+------------+-----------+------------+-----------------+
|5.1          |3.5         |1.4          |0.2         |Iris-setosa|0.0         |[5.1,3.5,1.4,0.2]|
|4.9          |3.0         |1.4          |0.2         |Iris-setosa|0.0         |[4.9,3.0,1.4,0.2]|
|4.7          |3.2         |1.3          |0.2         |Iris-setosa|0.0         |[4.7,3.2,1.3,0.2]|
|4.6          |3.1         |1.5          |0.2         |Iris-setosa|0.0         |[4.6,3.1,1.5,0.2]|
|5.0          |3.6         |1.4          |0.2         |Iris-setosa|0.0         |[5.0,3.6,1.4,0.2]|
|5.4          |3.9         |1.7          |0.4         |Iris-setosa|0.0         |[5.4,3.9,1.7,0.4]|
|4.6          |3.4         |1.4          |0.3         |Iris-setosa|0.0         |[4.6,3.4,1.4,0.3]|
|5.0      

In [ ]:
 # Selecionando atributos que interessam para o modelo

In [35]:
df_features = df_features.select(df_features.indexedLabel, df_features.features)
df_features.show(50, False)

+------------+-----------------+
|indexedLabel|features         |
+------------+-----------------+
|0.0         |[5.1,3.5,1.4,0.2]|
|0.0         |[4.9,3.0,1.4,0.2]|
|0.0         |[4.7,3.2,1.3,0.2]|
|0.0         |[4.6,3.1,1.5,0.2]|
|0.0         |[5.0,3.6,1.4,0.2]|
|0.0         |[5.4,3.9,1.7,0.4]|
|0.0         |[4.6,3.4,1.4,0.3]|
|0.0         |[5.0,3.4,1.5,0.2]|
|0.0         |[4.4,2.9,1.4,0.2]|
|0.0         |[4.9,3.1,1.5,0.1]|
|0.0         |[5.4,3.7,1.5,0.2]|
|0.0         |[4.8,3.4,1.6,0.2]|
|0.0         |[4.8,3.0,1.4,0.1]|
|0.0         |[4.3,3.0,1.1,0.1]|
|0.0         |[5.8,4.0,1.2,0.2]|
|0.0         |[5.7,4.4,1.5,0.4]|
|0.0         |[5.4,3.9,1.3,0.4]|
|0.0         |[5.1,3.5,1.4,0.3]|
|0.0         |[5.7,3.8,1.7,0.3]|
|0.0         |[5.1,3.8,1.5,0.3]|
|0.0         |[5.4,3.4,1.7,0.2]|
|0.0         |[5.1,3.7,1.5,0.4]|
|0.0         |[4.6,3.6,1.0,0.2]|
|0.0         |[5.1,3.3,1.7,0.5]|
|0.0         |[4.8,3.4,1.9,0.2]|
|0.0         |[5.0,3.0,1.6,0.2]|
|0.0         |[5.0,3.4,1.6,0.4]|
|0.0      

In [36]:
df_features.printSchema()

root
 |-- indexedLabel: double (nullable = false)
 |-- features: vector (nullable = true)



In [37]:
df_novo = df_features.withColumnRenamed("indexedLabel", "label")
df_novo.show()

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
|  0.0|[5.4,3.9,1.7,0.4]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[5.0,3.4,1.5,0.2]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.9,3.1,1.5,0.1]|
|  0.0|[5.4,3.7,1.5,0.2]|
|  0.0|[4.8,3.4,1.6,0.2]|
|  0.0|[4.8,3.0,1.4,0.1]|
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[5.8,4.0,1.2,0.2]|
|  0.0|[5.7,4.4,1.5,0.4]|
|  0.0|[5.4,3.9,1.3,0.4]|
|  0.0|[5.1,3.5,1.4,0.3]|
|  0.0|[5.7,3.8,1.7,0.3]|
|  0.0|[5.1,3.8,1.5,0.3]|
+-----+-----------------+
only showing top 20 rows



# Base para treino e teste

In [66]:
(trainingData, testData) = df_novo.randomSplit([0.8, 0.4])

print(trainingData.count(), testData.count())

103 47


In [67]:
trainingData.show(3)
testData.show(3)

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.4,3.0,1.3,0.2]|
+-----+-----------------+
only showing top 3 rows

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[4.5,2.3,1.3,0.3]|
|  0.0|[4.8,3.0,1.4,0.3]|
|  0.0|[4.8,3.4,1.6,0.2]|
+-----+-----------------+
only showing top 3 rows



## Regressão Logística

In [68]:
from pyspark.ml.classification import LogisticRegression #, BinaryLogisticRegressionTrainingSummary
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [69]:
lru = LogisticRegression()

In [70]:
lruModelo = lru.fit(trainingData)

In [71]:
resultado = lruModelo.summary
print("Accuracy: %s" % str(resultado.accuracy))
print("True positive: %s" % str(resultado.truePositiveRateByLabel))
print("False positive: %s" % str(resultado.falsePositiveRateByLabel))
print(resultado.precisionByLabel)

Accuracy: 0.9805825242718447
True positive: [1.0, 0.9696969696969697, 0.9696969696969697]
False positive: [0.0, 0.014285714285714285, 0.014285714285714285]
[1.0, 0.9696969696969697, 0.9696969696969697]


In [72]:
# Aplicando a base de teste no Modelo
df_resultado = lruModelo.transform(testData)
df_resultado.show(30,truncate=False)

+-----+-----------------+-----------------------------------------------------------+------------------------------------------------------------------+----------+
|label|features         |rawPrediction                                              |probability                                                       |prediction|
+-----+-----------------+-----------------------------------------------------------+------------------------------------------------------------------+----------+
|0.0  |[4.5,2.3,1.3,0.3]|[18.251758646151604,13.131865956067923,-31.383624602219523]|[0.9940588440780677,0.005941155921932273,2.760826034169026E-22]   |0.0       |
|0.0  |[4.8,3.0,1.4,0.3]|[26.007201131885676,10.676559535333023,-36.68376066721869] |[0.9999997802206649,2.197793350776315E-7,5.93828626961446E-28]    |0.0       |
|0.0  |[4.8,3.4,1.6,0.2]|[32.002113660146435,8.352564896662841,-40.35467855680927]  |[0.9999999999464042,5.359588819698459E-11,3.7656886722456565E-32] |0.0       |
|0.0  |[5.0,3.0,

In [73]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = df_resultado.select(['label']).collect()
y_pred = df_resultado.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        13
         1.0       1.00      0.94      0.97        17
         2.0       0.94      1.00      0.97        17

    accuracy                           0.98        47
   macro avg       0.98      0.98      0.98        47
weighted avg       0.98      0.98      0.98        47

